## ARQUITETURA

1) RAJPURKAR, Pranav et. al. Cardiologist-Level Arrhythmia Detection with Convolutional Neural Networks

Importando bibliotecas

In [1]:
#Bibliotecas básicas
import numpy as np
from numpy import load
import matplotlib.pyplot as plt

#Biblioteca para rede neural
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Dropout, BatchNormalization, Flatten, Dense, Activation, ReLU, Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

from tensorflow.keras import backend as K

Baixando os dados

In [2]:
data = np.load('dados.npz')

Criando os conjuntos de treinamento e de teste

In [3]:
#Conjunto de treinamento
X_train = data['X_train']
y_train = data['y_train']
#Conjunto de teste
X_test = data['X_test']
y_test = data['y_test']

Resumo

In [4]:
#Sumário
print("\tDIMENSÕES \n-------------------------")
print(f'X_train = {X_train.shape}')
print(f'y_train = {y_train.shape}')
print("-------------------------")
print(f'X_test = {X_test.shape}')
print(f'y_test = {y_test.shape}')
print("-------------------------")
print("\tSUMÁRIO \n-------------------------")
print(f"Total de exemplos de treinamento = {X_train.shape[0]}")
print(f"Total de exemplos de teste = {X_test.shape[0]}")
print(f"Total de amostras = {X_train.shape[1]}")

	DIMENSÕES 
-------------------------
X_train = (18358, 1000, 12)
y_train = (18358, 5)
-------------------------
X_test = (2055, 1000, 12)
y_test = (2055, 5)
-------------------------
	SUMÁRIO 
-------------------------
Total de exemplos de treinamento = 18358
Total de exemplos de teste = 2055
Total de amostras = 1000


Definindo dimensões

In [5]:
input_shape=(1000, 12)
print(f"Input shape = {input_shape}")

Input shape = (1000, 12)


**DESENVOLVIMENTO DA REDE**

In [6]:
rate_drop = 0.5

In [7]:
initializer = 'he_normal'

In [8]:
K.clear_session()

In [9]:
#Camada de entrada
input_1 = Input(shape=input_shape)

Primeiro bloco

In [10]:
num_filter = 64

conv_1 = Conv1D(kernel_size=16, strides=1, filters=num_filter,padding="same",kernel_initializer=initializer)(input_1)
bn_1 = BatchNormalization()(conv_1)
relu_1 = ReLU()(bn_1)

Primeiro bloco residual  
Stride = 2  
Filtros = 64

In [11]:
#Bloco residual
conv_2 = Conv1D(kernel_size=16,filters=num_filter,strides=1,padding="same",kernel_initializer=initializer)(relu_1)
bn_2 = BatchNormalization()(conv_2)
relu_2 = ReLU()(bn_2)
drop_1 = Dropout(rate_drop)(relu_2)
conv_3 = Conv1D(kernel_size = 16, filters=num_filter,strides=2,padding="same",kernel_initializer=initializer)(drop_1)

#Short connection
short_1 = MaxPooling1D(pool_size=1, strides=2)(relu_1)

#Adicionando camadas
add_1 = Add()([conv_3, short_1])

Segundo bloco residual  
Stride = 1  
Filtros = 64

In [12]:
#Bloco residual
bn_3 = BatchNormalization()(add_1)
relu_3 = ReLU()(bn_3)
drop_2 = Dropout(rate_drop)(relu_3)
conv_4 = Conv1D(kernel_size=16, filters=num_filter, strides=1, padding="same",kernel_initializer=initializer)(drop_2)
bn_4 = BatchNormalization()(conv_4)
relu_4 = ReLU()(bn_4)
drop_3 = Dropout(rate_drop)(relu_4)
conv_5 = Conv1D(kernel_size=16, filters=num_filter, strides=1, padding="same",kernel_initializer=initializer)(drop_3)

#Short connection
short_2 = MaxPooling1D(pool_size=1, strides=1)(add_1)

#Adicionando camadas
add_2 = Add()([conv_5, short_2])

Terceiro bloco residual  
Stride = 2  
Filtros = 64

In [13]:
#Bloco residual
bn_5 = BatchNormalization()(add_2)
relu_5 = ReLU()(bn_5)
drop_4 = Dropout(rate_drop)(relu_5)
conv_6 = Conv1D(kernel_size=16, filters=num_filter, strides=1, padding="same",kernel_initializer=initializer)(drop_4)
bn_6 = BatchNormalization()(conv_6)
relu_6 = ReLU()(bn_6)
drop_5 = Dropout(rate_drop)(relu_6)
conv_7 = Conv1D(kernel_size=16, filters=num_filter, strides=2, padding="same",kernel_initializer=initializer)(drop_5)

#Short connection
short_3 = MaxPooling1D(pool_size = 1, strides=2)(add_2)

#Adição dos blocos
add_3 = Add()([conv_7, short_3])

Quarto bloco residual  
Stride = 1  
Filtros = 64

In [14]:
#Bloco residual
bn_7 = BatchNormalization()(add_3)
relu_7 = ReLU()(bn_7)
drop_6 = Dropout(rate_drop)(relu_7)
conv_8 = Conv1D(kernel_size=16, filters=num_filter, strides=1, padding="same",kernel_initializer=initializer)(drop_6)
bn_8 = BatchNormalization()(conv_8)
relu_8 = ReLU()(bn_8)
drop_7 = Dropout(rate_drop)(relu_8)
conv_9 = Conv1D(kernel_size=16, filters=num_filter, strides=1, padding="same",kernel_initializer=initializer)(drop_7)

#Short connection
short_4 = MaxPooling1D(pool_size = 1, strides=1)(add_3)

#Adição dos blocos
add_4 = Add()([conv_9, short_4])

Quinto bloco residual  
Stride = 2  
Filtros = 128

In [15]:
num_filter = 128

#Bloco residual
bn_9 = BatchNormalization()(add_4)
relu_9 = ReLU()(bn_9)
drop_8 = Dropout(rate_drop)(relu_9)
conv_10 = Conv1D(kernel_size=16, filters=num_filter, strides=1, padding="same",kernel_initializer=initializer)(drop_8)
bn_10 = BatchNormalization()(conv_10)
relu_10 = ReLU()(bn_10)
drop_9 = Dropout(rate_drop)(relu_10)
conv_11 = Conv1D(kernel_size=16, filters=num_filter, strides=2, padding="same",kernel_initializer=initializer)(drop_9)

#Short connection
conv_aj5 = Conv1D(kernel_size=16,filters=num_filter,strides=1,padding="same")(add_4) #Ajustar o número de filtros para adicionar depois o Max ao conv_11
short_5 = MaxPooling1D(pool_size = 1, strides=2)(conv_aj5)

#Adição dos blocos
add_5 = Add()([conv_11, short_5])

Sexto bloco residual  
Stride = 1  
Filtros = 128

In [16]:
#Bloco residual
bn_11 = BatchNormalization()(add_5)
relu_11 = ReLU()(bn_11)
drop_9 = Dropout(rate_drop)(relu_11)
conv_12 = Conv1D(kernel_size=16, filters=num_filter, strides=1, padding="same",kernel_initializer=initializer)(drop_9)
bn_12 = BatchNormalization()(conv_12)
relu_12 = ReLU()(bn_12)
drop_10 = Dropout(rate_drop)(relu_12)
conv_13 = Conv1D(kernel_size=16, filters=num_filter, strides=1, padding="same",kernel_initializer=initializer)(drop_10)

#Short connection
short_6 = MaxPooling1D(pool_size = 1, strides=1)(add_5)

#Adição dos blocos
add_6 = Add()([conv_13, short_6])

Sétimo bloco residual  
Strides = 2  
Filtros = 128

In [17]:
#Bloco residual
bn_13 = BatchNormalization()(add_6)
relu_13 = ReLU()(bn_13)
drop_11 = Dropout(rate_drop)(relu_13)
conv_14 = Conv1D(kernel_size=16, filters=num_filter, strides=1, padding="same",kernel_initializer=initializer)(drop_11)
bn_14 = BatchNormalization()(conv_14)
relu_14 = ReLU()(bn_14)
drop_12 = Dropout(rate_drop)(relu_14)
conv_15 = Conv1D(kernel_size=16, filters=num_filter, strides=2, padding="same",kernel_initializer=initializer)(drop_12)

#Short connection
short_7 = MaxPooling1D(pool_size = 1, strides=2)(add_6)

#Adição dos blocos
add_7 = Add()([conv_15, short_7])

Oitavo bloco residual  
Stride = 1  
Filtros = 128

In [18]:
#Bloco residual
bn_15 = BatchNormalization()(add_7)
relu_15 = ReLU()(bn_15)
drop_13 = Dropout(rate_drop)(relu_15)
conv_16 = Conv1D(kernel_size=16, filters=num_filter, strides=1, padding="same",kernel_initializer=initializer)(drop_13)
bn_16 = BatchNormalization()(conv_16)
relu_16 = ReLU()(bn_16)
drop_14 = Dropout(rate_drop)(relu_16)
conv_17 = Conv1D(kernel_size=16, filters=num_filter, strides=1, padding="same",kernel_initializer=initializer)(drop_14)

#Short connection
short_8 = MaxPooling1D(pool_size = 1, strides=1)(add_7)

#Adição dos blocos
add_8 = Add()([conv_17, short_8])

Nono bloco residual  
Stride = 2  
Filtros = 192

In [19]:
num_filter = 192

#Bloco residual
bn_17 = BatchNormalization()(add_8)
relu_17 = ReLU()(bn_17)
drop_15 = Dropout(rate_drop)(relu_17)
conv_18 = Conv1D(kernel_size=16, filters=num_filter, strides=1, padding="same",kernel_initializer=initializer)(drop_15)
bn_18 = BatchNormalization()(conv_18)
relu_18 = ReLU()(bn_18)
drop_16 = Dropout(rate_drop)(relu_18)
conv_19 = Conv1D(kernel_size=16, filters=num_filter, strides=2, padding="same")(drop_16)

#Short connection
conv_aj17 = Conv1D(kernel_size=16, filters=num_filter, strides=1, padding="same")(add_8)
short_9 = MaxPooling1D(pool_size = 1, strides=2)(conv_aj17)

#Adição dos blocos
add_9 = Add()([conv_19, short_9])

Décimo bloco residual  
Stride = 1  
Filtros = 192

In [20]:
#Bloco residual
bn_19 = BatchNormalization()(add_9)
relu_19 = ReLU()(bn_19)
drop_17 = Dropout(rate_drop)(relu_19)
conv_20 = Conv1D(kernel_size=16, filters=num_filter, strides=1, padding="same",kernel_initializer=initializer)(drop_17)
bn_20 = BatchNormalization()(conv_20)
relu_20 = ReLU()(bn_20)
drop_18 = Dropout(rate_drop)(relu_20)
conv_21 = Conv1D(kernel_size=16, filters=num_filter, strides=1, padding="same",kernel_initializer=initializer)(drop_18)

#Short connection
short_10 = MaxPooling1D(pool_size = 1, strides=1)(add_9)

#Adição dos blocos
add_10 = Add()([conv_21, short_10])

Décimo primeiro bloco residual  
Stride = 2  
Filtros = 192

In [21]:
#Bloco residual
bn_21 = BatchNormalization()(add_10)
relu_21 = ReLU()(bn_21)
drop_19 = Dropout(rate_drop)(relu_21)
conv_22 = Conv1D(kernel_size=16, filters=num_filter, strides=1, padding="same",kernel_initializer=initializer)(drop_19)
bn_22 = BatchNormalization()(conv_22)
relu_22 = ReLU()(bn_22)
drop_20 = Dropout(rate_drop)(relu_22)
conv_23 = Conv1D(kernel_size=16, filters=num_filter, strides=2, padding="same",kernel_initializer=initializer)(drop_20)

#Short connection
short_11 = MaxPooling1D(pool_size = 1, strides=2)(add_10)

#Adição dos blocos
add_11 = Add()([conv_23, short_11])

Décimo segundo bloco residual  
Strides = 1  
Filtros = 192

In [22]:
#Bloco residual
bn_23 = BatchNormalization()(add_11)
relu_23 = ReLU()(bn_23)
drop_21 = Dropout(rate_drop)(relu_23)
conv_24 = Conv1D(kernel_size=16, filters=num_filter, strides=1, padding="same",kernel_initializer=initializer)(drop_21)
bn_24 = BatchNormalization()(conv_24)
relu_24 = ReLU()(bn_24)
drop_22 = Dropout(rate_drop)(relu_24)
conv_25 = Conv1D(kernel_size=16, filters=num_filter, strides=1, padding="same",kernel_initializer=initializer)(drop_22)

#Short connection
short_12 = MaxPooling1D(pool_size = 1, strides=1)(add_11)

#Adição dos blocos
add_12 = Add()([conv_25, short_12])

Décimo terceiro bloco residual  
Strides = 2  
Filtros = 256

In [23]:
num_filter = 256

#Bloco residual
bn_25 = BatchNormalization()(add_12)
relu_25 = ReLU()(bn_25)
drop_23 = Dropout(rate_drop)(relu_25)
conv_26 = Conv1D(kernel_size=16, filters=num_filter, strides=1, padding="same",kernel_initializer=initializer)(drop_23)
bn_26 = BatchNormalization()(conv_26)
relu_26 = ReLU()(bn_26)
drop_24 = Dropout(rate_drop)(relu_26)
conv_27 = Conv1D(kernel_size=16, filters=num_filter, strides=2, padding="same",kernel_initializer=initializer)(drop_24)

#Short connection
conv_aj27 = Conv1D(kernel_size=16, filters=num_filter, strides=1, padding="same")(add_12)
short_13 = MaxPooling1D(pool_size = 1, strides=2)(conv_aj27)

#Adição dos blocos
add_13 = Add()([conv_27, short_13])

Décimo quarto bloco residual  
Stride = 1  
Filtros = 256

In [24]:
#Bloco residual
bn_27 = BatchNormalization()(add_13)
relu_27 = ReLU()(bn_27)
drop_25 = Dropout(rate_drop)(relu_27)
conv_28 = Conv1D(kernel_size=16, filters=num_filter, strides=1, padding="same",kernel_initializer=initializer)(drop_25)
bn_28 = BatchNormalization()(conv_28)
relu_28 = ReLU()(bn_28)
drop_26 = Dropout(rate_drop)(relu_28)
conv_29 = Conv1D(kernel_size=16, filters=num_filter, strides=1, padding="same",kernel_initializer=initializer)(drop_26)

#Short connection
short_14 = MaxPooling1D(pool_size = 1, strides=1)(add_13)

#Adição dos blocos
add_14 = Add()([conv_29, short_14])

Décimo quinto bloco residual  
Stride = 2  
Filtros = 256

In [25]:
#Bloco residual
bn_29 = BatchNormalization()(add_14)
relu_29 = ReLU()(bn_29)
drop_27 = Dropout(rate_drop)(relu_29)
conv_30 = Conv1D(kernel_size=16, filters=num_filter, strides=1, padding="same",kernel_initializer=initializer)(drop_27)
bn_30 = BatchNormalization()(conv_30)
relu_30 = ReLU()(bn_30)
drop_28 = Dropout(rate_drop)(relu_30)
conv_31 = Conv1D(kernel_size=16, filters=num_filter, strides=2, padding="same",kernel_initializer=initializer)(drop_28)

#Short connection
short_15 = MaxPooling1D(pool_size = 1, strides=2)(add_14)

#Adição dos blocos
add_15 = Add()([conv_31, short_15])

Décimo sexto bloco residual  
Stride = 1  
Filtros = 256

In [26]:
#Bloco residual
bn_31 = BatchNormalization()(add_15)
relu_31 = ReLU()(bn_31)
drop_29 = Dropout(rate_drop)(relu_31)
conv_32 = Conv1D(kernel_size=16, filters=num_filter, strides=1, padding="same",kernel_initializer=initializer)(drop_29)
bn_32 = BatchNormalization()(conv_32)
relu_32 = ReLU()(bn_32)
drop_30 = Dropout(rate_drop)(relu_32)
conv_33 = Conv1D(kernel_size=16, filters=num_filter, strides=1, padding="same",kernel_initializer=initializer)(drop_30)

#Short connection
short_16 = MaxPooling1D(pool_size = 1, strides=1)(add_15)

#Adição dos blocos
add_16 = Add()([conv_33, short_16])

Último bloco da rede

In [27]:
#Últimas camadas
bn_33 = BatchNormalization()(add_16)
relu_33 = ReLU()(bn_33)
flat = Flatten()(relu_33)
dense_1 = Dense(32)(flat)
classification = Dense(5, activation='sigmoid')(dense_1)

In [28]:
#Criando modelo
model = Model(inputs=input_1,outputs=classification)
# model.summary()

**TREINANDO O MODELO**

In [29]:
#Parâmetros de otimização
lr = 0.001
epochs = 10
batch_size = 32

In [30]:
#Otimizador Adam
opt = Adam(lr, beta_1=0.9, beta_2=0.999)

In [31]:
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [32]:
callbacks = []

#Reduz a taxa de aprendizagem quando o erro de validação para de melhorar
callbacks.append(ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=lr/10000))

In [33]:
#Treino do modelo
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), callbacks=callbacks)

Train on 18358 samples, validate on 2055 samples
Epoch 1/10
18358/18358 [==============================] - 146s 8ms/sample - loss: 0.3885 - accuracy: 0.8315 - val_loss: 0.5447 - val_accuracy: 0.8484
Epoch 2/10
18358/18358 [==============================] - 108s 6ms/sample - loss: 0.3160 - accuracy: 0.8658 - val_loss: 0.3266 - val_accuracy: 0.8702
Epoch 3/10
18358/18358 [==============================] - 108s 6ms/sample - loss: 0.2969 - accuracy: 0.8753 - val_loss: 0.3176 - val_accuracy: 0.8672
Epoch 4/10
18358/18358 [==============================] - 108s 6ms/sample - loss: 0.2827 - accuracy: 0.8829 - val_loss: 0.4349 - val_accuracy: 0.8203
Epoch 5/10
18358/18358 [==============================] - 107s 6ms/sample - loss: 0.2741 - accuracy: 0.8864 - val_loss: 0.3105 - val_accuracy: 0.8813
Epoch 6/10
18358/18358 [==============================] - 108s 6ms/sample - loss: 0.2692 - accuracy: 0.8886 - val_loss: 0.2998 - val_accuracy: 0.8808
Epoch 7/10
18358/18358 [===========================

**AVALIANDO O MODELO**

In [ ]:
loss_train = history.history['loss']
accuracy_train = history.history['accuracy']

loss_val = history.history['val_loss']
accuracy_val = history.history['val_accuracy']

epochs = history.epoch

In [ ]:
plt.plot(epochs, loss_train, '-o')
plt.plot(epochs, loss_val, '-*')

plt.title('Custo de treinamento e de validação')
plt.xlabel('Epochs')
plt.ylabel('Custo')
plt.legend(['Custo de treino', 'Custo de validação'])
local = 'E:/Usuários/Sarah/Documentos/UTFPR/TCC/Resultados/Gráficos/Custo_atual_100'
plt.savefig(local)
plt.show()

In [ ]:
plt.plot(epochs, accuracy_train, '-o')
plt.plot(epochs, accuracy_val, '-*')

plt.title('Acurácia de treino e de validação')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend(['Acurácia de treino', 'Acurácia de validação'])
local = 'E:/Usuários/Sarah/Documentos/UTFPR/TCC/Resultados/Gráficos/Acuracia_atual_100'
plt.savefig(local)
plt.show()

In [ ]:
score = model.evaluate(X_test, y_test)
print(f"Custo de teste = {score[0]:.4f}")
print(f"Acurácia de teste = {100*score[1]:.2f}%")

**Predições**

classification = ['NORM', 'MI', 'CD', 'STTC', 'HYP']

In [ ]:
prediction = model.predict(X_test) #Realiza a predição das probabilidades de cada label
prediction_bin = np.array(prediction)

In [ ]:
print(prediction[0])
print(prediction_bin[0])

Transforma o vetor prediction que consta as probabilidades para cada label em um vetor binário (0 ou 1)

In [ ]:
#Dada as probabilidades de cada label, se maior ou igual que 0.5, é considerada a classe de diagnóstico
for indice in range(prediction_bin.shape[0]):
    for i in range(prediction_bin.shape[1]):
        if prediction_bin[indice][i] >= 0.5:
            prediction_bin[indice][i] = 1
        else:
            prediction_bin[indice][i] = 0
prediction_bin = np.array(prediction_bin,dtype='int')

In [ ]:
print(f'Previsão: {prediction_bin[23]} \t\tLabel real: {y_test[23]}')

Transformar o vetor binario em strings  
https://medium.com/deep-learning-with-keras/how-to-solve-multi-label-classification-problems-in-deep-learning-with-tensorflow-keras-7fb933243595  
https://machinelearningmastery.com/multi-label-classification-with-deep-learning/

In [ ]:
label_string = ['NORM','MI','CD','STTC','HYP']

In [ ]:
def get_strings(label_string,label_bin):  #label_bin pode ser de teste ou predições
    label_bin_string = []
    for x in range(len(label_bin)):
        lst = []
        for y in range(len(label_string)):
            value = label_bin[x][y]
            if value == 1:
                lst.append(label_string[y])
        label_bin_string.append(lst)
    
    return label_bin_string

In [ ]:
#Criando os vetores de label strings do vetor y_test
y_test_string = get_strings(label_string,y_test)

In [ ]:
#Criando os vetores de label strings do vetor y_test
prediction_string = get_strings(label_string,prediction_bin)

In [ ]:
print(y_test_string[23])
print(prediction_string[23])

Visualizando as predições

In [ ]:
index = 23
print(f'Index: {index}\n')

print(f'Diagnóstico = {y_test[index]}')
print(f'Predição = {prediction_bin[index]}\n')

print(f'Diagnóstico = {y_test_string[index]}')
print(f'Predição = {prediction_string[index]}')

In [ ]:
#Verificar a acc do exemplo correspondente ao índice!
from sklearn.metrics import accuracy_score

In [ ]:
acc_index = accuracy_score(y_test[index],prediction_bin[index])
print(f'Acurácia do exemplo = {100 * acc_index:.2f}%')

In [ ]:
valor_med = X_test[index, ].mean(axis=-1)
fig_s, ax_s = plt.subplots(figsize=(10,7))
ax_s.set_title(f'Diagnóstico: {y_test_string[index]}       Predição:{prediction_string[index]}')
ax_s.plot(valor_med)
plt.show()

Matriz de confusão

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report
import pandas as pd
import seaborn as sns

**Precision**  
Precision can be seen as a measure of a classifier’s exactness. For each class, it is defined as the ratio of true positives to the sum of true and false positives. Said another way, “for all instances classified positive, what percent was correct?”  

**Recall**  
Recall is a measure of the classifier’s completeness; the ability of a classifier to correctly find all positive instances. For each class, it is defined as the ratio of true positives to the sum of true positives and false negatives. Said another way, “for all instances that were actually positive, what percent was classified correctly?”  

**F1 score**  
The F1 score is a weighted harmonic mean of precision and recall such that the best score is 1.0 and the worst is 0.0. Generally speaking, F1 scores are lower than accuracy measures as they embed precision and recall into their computation. As a rule of thumb, the weighted average of F1 should be used to compare classifier models, not global accuracy.  

**Support**  
Support is the number of actual occurrences of the class in the specified dataset. Imbalanced support in the training data may indicate structural weaknesses in the reported scores of the classifier and could indicate the need for stratified sampling or rebalancing. Support doesn’t change between models but instead diagnoses the evaluation process.  

https://www.scikit-yb.org/en/latest/api/classifier/classification_report.html#:~:text=not%20global%20accuracy.-,support,for%20stratified%20sampling%20or%20rebalancing.

In [ ]:
report = classification_report(y_test,prediction_bin,output_dict=True,target_names=['NORM', 'MI', 'CD', 'STTC', 'HYP'])

In [ ]:
report_df = pd.DataFrame.from_dict(report, orient='index')

In [ ]:
report_df

In [ ]:
cm = multilabel_confusion_matrix(y_test, prediction_bin)

In [ ]:
# https://www.kaggle.com/code/trolukovich/multi-label-classification-keras/notebook
# Plot confusion matrix 
fig = plt.figure(figsize = (14, 8))
for i, (label, matrix) in enumerate(zip(label_string, cm)):
    plt.subplot(f'23{i+1}')
    labels = [f'Não {label}', label]
    sns.heatmap(matrix, annot = True, square = True, fmt = 'd', cbar = False, cmap = 'Blues', 
                xticklabels = labels, yticklabels = labels, linecolor = 'black', linewidth = 1)
    plt.title(label)

plt.tight_layout()
plt.show()